In [2]:
import numpy as np

def convTranspose(input, kernel, stride, padding, output_size):
    input_height, input_width = input.shape
    kernel_height, kernel_width = kernel.shape
    output_height, output_width = output_size

    output = np.zeros(output_size)

    for i in range(output_height):
        for j in range(output_width):
            for k in range(kernel_height):
                for l in range(kernel_width):
                    row = i * stride + k - padding
                    col = j * stride + l - padding

                    if row >= 0 and row < input_height and col >= 0 and col < input_width:
                        output[i, j] += input[row, col] * kernel[k, l]

    return output

In [4]:
input = np.array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 9]])

kernel = np.array([[1, 1],
                   [1, 1]])

stride = 1
padding = 1
output_size = (4, 4)
torch_conv = torch.nn.functional.conv_transpose2d(torch.Tensor([[input]]), torch.Tensor([[kernel]]))
my_conv = convTranspose(input, kernel, stride = 1, padding = 1, output_size = (4,4))
print("Результат работы исходной функции: \n" , np.array(torch_conv))
print("Результат работы написанной функции: \n", my_conv)
print(np.allclose(my_conv, np.array(torch_conv)))

Результат работы исходной функции: 
 [[[[ 1.  3.  5.  3.]
   [ 5. 12. 16.  9.]
   [11. 24. 28. 15.]
   [ 7. 15. 17.  9.]]]]
Результат работы написанной функции: 
 [[ 1.  3.  5.  3.]
 [ 5. 12. 16.  9.]
 [11. 24. 28. 15.]
 [ 7. 15. 17.  9.]]
True


In [105]:
import numpy as np

def transposed_conv2d(in_tensor, weight, bias=None, stride=1, padding=0, dilation=1):
    batch_size, in_channels, in_height, in_width = in_tensor.shape
    in_channels, out_channels, kernel_height, kernel_width = weight.shape

    out_height = (in_height - 1) * stride - 2 * padding + dilation * (kernel_height - 1) + 1
    out_width = (in_width - 1) * stride - 2 * padding + dilation * (kernel_width - 1) + 1
    out_tensor = np.zeros((batch_size, out_channels, out_height, out_width))

    for b in range(batch_size):
        for i in range(in_channels):
            for o in range(out_channels):
                for h in range(in_height):
                    for w in range(in_width):
                        out_tensor[b, o, h*stride:h*stride+kernel_height, w*stride:w*stride+kernel_width] += in_tensor[b, i, h, w] * weight[i, o]

    if bias is not None:
        out_tensor += bias.reshape(1, -1, 1, 1)

    return out_tensor

In [106]:
import torch

input_tensor = np.array([[[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]]])
filters = np.array([[[[1, 1], [1, 1]]]])

torch_conv = torch.nn.functional.conv_transpose2d(torch.Tensor(input_tensor), torch.Tensor(filters))
my_conv = transposed_conv2d(input_tensor, filters)
print("Результат работы исходной функции: \n" , np.array(torch_conv))
print("Результат работы написанной функции: \n", my_conv)
print(np.allclose(my_conv, np.array(torch_conv)))

Результат работы исходной функции: 
 [[[[ 1.  3.  5.  7.  4.]
   [ 6. 14. 18. 22. 12.]
   [14. 30. 34. 38. 20.]
   [22. 46. 50. 54. 28.]
   [13. 27. 29. 31. 16.]]]]
Результат работы написанной функции: 
 [[[[ 1.  3.  5.  7.  4.]
   [ 6. 14. 18. 22. 12.]
   [14. 30. 34. 38. 20.]
   [22. 46. 50. 54. 28.]
   [13. 27. 29. 31. 16.]]]]
True
